In [1]:
%lsmagic

Available line magics:
%alias  %alias_magic  %autocall  %automagic  %autosave  %bookmark  %cd  %clear  %cls  %colors  %config  %connect_info  %copy  %ddir  %debug  %dhist  %dirs  %doctest_mode  %echo  %ed  %edit  %env  %gui  %hist  %history  %killbgscripts  %ldir  %less  %load  %load_ext  %loadpy  %logoff  %logon  %logstart  %logstate  %logstop  %ls  %lsmagic  %macro  %magic  %matplotlib  %mkdir  %more  %notebook  %page  %pastebin  %pdb  %pdef  %pdoc  %pfile  %pinfo  %pinfo2  %popd  %pprint  %precision  %profile  %prun  %psearch  %psource  %pushd  %pwd  %pycat  %pylab  %qtconsole  %quickref  %recall  %rehashx  %reload_ext  %ren  %rep  %rerun  %reset  %reset_selective  %rmdir  %run  %save  %sc  %set_env  %store  %sx  %system  %tb  %time  %timeit  %unalias  %unload_ext  %who  %who_ls  %whos  %xdel  %xmode

Available cell magics:
%%!  %%HTML  %%SVG  %%bash  %%capture  %%cmd  %%debug  %%file  %%html  %%javascript  %%js  %%latex  %%perl  %%prun  %%pypy  %%python  %%python2  %%python3  %%rub

In [2]:
%matplotlib inline

In [3]:
import sys
import os
import tempfile

import pandas as pd

import weka.core.jvm as jvm
import pygraphviz
import PIL
from PIL import Image
from weka.core.converters import Loader
# import weka.core.ContingencyTables.entropy
from weka.classifiers import Classifier, PredictionOutput, Evaluation
import weka.plot.graph as graph  # NB: pygraphviz and PIL are required
from weka.core.converters import Loader, Saver
import traceback
import weka.core.serialization as serialization
import weka.plot.graph as plot_graph
import weka.plot.classifiers as plot_cls

# Function

In [4]:
def load_data(data_train, data_testing):
    
    loader = Loader(classname="weka.core.converters.ArffLoader")
    train = loader.load_file(data_train + ".arff")
    testing = loader.load_file(data_testing + ".arff")
    
    
    train.class_is_last()
    testing.class_is_last()
    train.class_index = train.num_attributes - 1
#     print testing.classAttribute()
    
    # print(data)
    #print "Data Train :", len(train)
    # print(train)
    #print "Data Test :", len(testing)
    # print(test)
    
    return train,testing

def build_tree(train, testing, directory):

    cls = Classifier(classname="weka.classifiers.trees.J48")
    cls.options = ["-U"] 
    cls.build_classifier(train)
    
    #print model
#     print(cls) 

    
    #### save classifier
    

    outfile = directory + "Unpruned_j48.model"
    serialization.write(outfile, cls)
    f = open( directory + "Unpruned_tree.txt", "w+" )
    f.write('{}'.format(cls))
    f.close()
#     print cls
    graph.plot_dot_graph(cls.graph, filename = directory + "Unpruned_Result_tree.png")
#     print cls.graph
    return cls


def testing_evaluation(cls, train, testing, directory):
     # Testing
    list_summ = []
#     evaluation = Evaluation(train, cost_matrix = None)
    evaluation = Evaluation(train, cost_matrix = None)
    evaluation.test_model(cls, testing)
#     print(evaluation.confusion_matrix)
    print(evaluation.class_details())
#     print evaluation
    print evaluation.summary(title="Summary", complexity=False)

    # Output predictions
    cols = ['Actual', 'Predicted', 'Error', 'Distribution']
    list_inst =[]
    df =pd.DataFrame(columns=cols)
   
    for index, inst in enumerate(testing):
        pred = cls.classify_instance(inst)
        dist = cls.distribution_for_instance(inst)
        myFormattedList = [ '%.2f' % elem for elem in dist.tolist() ]
        
        

        list_inst.append({'Actual':inst.get_string_value(inst.class_index),
                          'Predicted': inst.class_attribute.value(int(pred)),
                          'Error':"yes" if pred != inst.get_value(inst.class_index)else "no",
                          'Distribution' : myFormattedList,
                         })  

    df = df.append(list_inst)
    df = df.reset_index(drop=True)
    df.to_csv(directory + 'Unpruned_TestingResult.csv', index=False)
    summ = summary(directory, evaluation, list_summ, 0)
    
    
    return df,evaluation,summ

    
    
def summary(directory, evaluation, list_summ, confval):
    
    sum_item = evaluation.summary().split()
    precision = evaluation.precision(1)
  
#     print inst.class_attribute.value(recall)
#     print len(evaluation.recall())
#     print evaluation
#     print("mean_prior entropy 1", evaluation.sf_mean_prior_entropy)
    
    class_detail  =evaluation.class_details()
    class_detail_item = class_detail.split()
    len_detail = len(class_detail_item)
    sum_item.append(0)
    sum_item.append(confval)
    
    #     precision + recall
 
    
    sum_item.append(class_detail_item[21]+"-"+ class_detail_item[27])
    sum_item.append(class_detail_item[22] + "-"+ class_detail_item[27])

    
    sum_item.append(class_detail_item[30]+ "-"+class_detail_item[36])
    sum_item.append(class_detail_item[31] +"-"+ class_detail_item[36])
    
    sum_item.append(class_detail_item[39] + "-"+class_detail_item[45])
    sum_item.append(class_detail_item[40] +"-"+ class_detail_item[45])
    
    if len_detail == 65 :
        sum_item.append(class_detail_item[48] +"-"+ class_detail_item[54])
        sum_item.append(class_detail_item[49] +"-"+ class_detail_item[54])
#     print (class_detail_item[21],class_detail_item[30], class_detail_item[39])

    
    
    list_summ.append(sum_item)
    
    conf_matrix = evaluation.matrix()
    
    
#     print type(class_detail_item)
#     print class_detail_item 
#     print class_detail_item
    f = open( directory+str(confval)+"Confusion Matrix.txt", "w+" )
    f.write('{} \n {}'.format(conf_matrix, class_detail))
    f.close()
   
    
    return list_summ
    
def pruning_tree(directory, training, testing, list_summ, summary_):

    ######  Recursive Pruning
    for value in range(50,1,-1):

        confval = float(value)/100
        confval = str(confval)
        
        cls = Classifier(classname="weka.classifiers.trees.J48")
        cls.options = ["-C", confval]
        
        
        #### cls.options = ["-R"]
          
        print "__________Pruning tree with Confidence Value : ", confval
        print(cls.options)

        
        cls.build_classifier(training)
        
        outfile = directory + confval+ "pruning_j48.model"
        serialization.write(outfile, cls)
        f = open( directory + confval+"_pruning_tree.txt", "w+" )
        f.write('{}'.format(cls))
        f.close()
#         graph.plot_dot_graph(cls.graph, filename = directory +confval+"pruning_j48.png")

        #### Testing
        evaluation = Evaluation(training, cost_matrix = None)
        evaluation.test_model(cls, testing)
        
        summ = summary(directory, evaluation,list_summ,confval)
        
    summary_ = summary_.append(summ)
    summary_ = summary_.reset_index(drop =  True)
    summary_.to_csv(directory +'summ_temp.csv', index=False)
    
    return summary_


def re_summary(summary, directory):
    cols_new = ['Confidence Value','Correctly Classified', 'Incorrectly Classified', 'Kappa', 'MEE',
            'RMSE','Relative abs. error', 'Root Relative abs. error']
    new_summary = pd.DataFrame(columns=cols_new)

    #Assign Summary
    new_summary['Confidence Value'] = summary.iloc[:,41]
    new_summary['precision1'] = summary.iloc[:,42]
    new_summary['recall1'] = summary.iloc[:,43]
    
    new_summary['precision2'] = summary.iloc[:,44]
    new_summary['recall2'] = summary.iloc[:,45]
    
    new_summary['precision3'] = summary.iloc[:,46]
    new_summary['recall3'] = summary.iloc[:,47]
    
    if len(summary.columns) == 50 :
        new_summary['precision4'] = summary.iloc[:,48]
        new_summary['recall4'] = summary.iloc[:,49]

    new_summary['Correctly Classified'] = summary.iloc[:,4]
    new_summary['Incorrectly Classified'] = summary.iloc[:,10]
    new_summary['Kappa'] = summary.iloc[:,14]
    new_summary['MEE'] = summary.iloc[:,18]
    new_summary['RMSE'] = summary.iloc[:,23]
    new_summary['Relative abs. error'] = summary.iloc[:,27]
    new_summary['Root Relative abs. error'] = summary.iloc[:,33]
    
#     new_summary = new_summary[].astype('float64')
    new_summary = new_summary.sort_values(by=['Confidence Value'], ascending=True)

    new_summary = new_summary.reset_index(drop=True)
    new_summary.to_csv( directory + 'Summary.csv')
    return new_summary
#     print display(new_summary)

    
def data_demografis(data,cls):
    datax = pd.DataFrame(columns = data.columns)
    if cls == "1": #IPK
        datax =data['JUR','JK','GAJI_ORTU','PDK_IBU','PEK_IBU','PEK_AYAH','IPK_TERAKHIR']
        datax.to_csv("DataResult/final/uji/data_uji_mipa_ipk_demografis")
    if cls == "2":#STATUS LULUS
        datax = data['JUR','JK','GAJI_ORTU','PDK_IBU','PEK_IBU','PEK_AYAH', 'STATUS_LULUS']
    


# Main Function

In [ ]:
def main(args):
    
    list_summ = []
    summary = pd.DataFrame(columns = None)
    cols_summary = ['A','B','C','D','E','F','G','H','I']
    
    #1. Data Mipa kelas Ipk
    ##################################################
    
    print("Data mipa kelas ipk")
    data_train = "DataPreprocessing/data_mipa_ipk"
    data_testing = "DataPreprocessing/data_uji_mipa_ipk"
    directory = "DataResult/original/mipa-ipk/"
   
    
    train,testing = load_data(data_train, data_testing)
#     print(testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory, train, testing, temp, summary)
    re_summary(summary_, directory)
    
    
    #2. Data Mipa kelas Lulus
    ##################################################
    
    print("Data mipa kelas lulus")
    
    data_train = "DataPreprocessing/data_mipa_lulus"
    data_testing = "DataPreprocessing/data_uji_mipa_lulus"
    directory = "DataResult/original/mipa-lulus/"
    
    train,testing = load_data(data_train, data_testing)
   
    cls = build_tree(train,testing,directory)

    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory, train, testing, temp, summary)
    re_summary(summary_, directory)
    


    #3. Data sosial kelas ipk
    ##################################################
    
    print("Data sosial kelas ipk")
    
    data_train = "DataPreprocessing/data_sosial_ipk"
    data_testing = "DataPreprocessing/data_uji_sosial_ipk"
    directory = "DataResult/original/sosial-ipk/"
       
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   
  
    
    
#     4. Data sosial kelas lulus
    ##################################################
    
    print("Data sosial kelas lulus")
    
    data_train = "DataPreprocessing/data_sosial_lulus"
    data_testing = "DataPreprocessing/data_uji_sosial_lulus"
    directory = "DataResult/original/sosial-lulus/"
   

    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   



     ################  oversampling   ##################
    

    
    print "-------------Oversampling------------"
    
    #5.Data mipa kelas ipk - oversampling
    ##################################################
    
    print("Data mipa kelas ipk - oversampling")
    
    data_train = "DataPreprocessing/data_mipa_ipk_over"
    data_testing = "DataPreprocessing/data_uji_mipa_ipk"
    directory = "DataResult/oversampling/mipa-ipk/"
   
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
    
    
    
    
    #6. Data mipa kelas lulus - oversampling
     ##################################################
        
    print("Data mipa kelas lulus - oversampling")
    
    
    
    data_train = "DataPreprocessing/data_mipa_lulus_over"
    data_testing = "DataPreprocessing/data_uji_mipa_lulus"
    directory = "DataResult/oversampling/mipa-lulus/"
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
    
    
    #7. Data Sosial kelas Ipk - oversampling
    ###################################################
    
    print("Data sosial kelas ipk - oversampling")
    
    data_train = "DataPreprocessing/data_sosial_ipk_over"
    data_testing = "DataPreprocessing/data_uji_sosial_ipk"
    directory = "DataResult/oversampling/sosial-ipk/"
    
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   

    #8. Data Sosial kelas Lulus - Oversampling     
    ##################################################
    
    print("Data Sosial kelas lulus - oversampling")
    
    data_train = "DataPreprocessing/data_sosial_lulus_over"
    data_testing = "DataPreprocessing/data_uji_sosial_lulus"
    directory = "DataResult/oversampling/sosial-lulus/"
   
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   

    #####################  Pure Random Data Uji  #####################
    print("---------------------------------------Random data Uji---------------------------------------")
    
    
    #9. Data Mipa kelas IPK - Data Uji Random 
    ##################################################
    
    print("Data mipa kelas ipk - Data Uji Random") 
    
    data_train = "DataPreprocessing/data_mipa_ipk_random"
    data_testing = "DataPreprocessing/data_uji_mipa_ipk_random"
    directory = "DataResult/random/mipa-ipk/"
   
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
    
    #10. Data Mipa kelas lulus - Data Uji Random 
    ##################################################    
    
    print("Data mipa kelas lulus - Data Uji Random")
    
    data_train = "DataPreprocessing/data_mipa_lulus_random"
    data_testing = "DataPreprocessing/data_uji_mipa_lulus_random"
    directory = "DataResult/random/mipa-lulus/"
   
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
    
   
    #11. Data Sosial kelas ipk - Data Uji Random 
    ##################################################  
    
    print("Data sosial kelas ipk - data uji random")
    
    data_train = "DataPreprocessing/data_sosial_ipk_random"
    data_testing = "DataPreprocessing/data_uji_sosial_ipk_random"
    directory = "DataResult/random/sosial-ipk/"
   
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   

    #12. Data Sosial kelas lulus - Data Uji Random 
    ##################################################
    
    print("Data sosial kelas lulus - Data Uji Random")
    
    data_train = "DataPreprocessing/data_sosial_lulus_random"
    data_testing = "DataPreprocessing/data_uji_sosial_lulus_random"
    directory = "DataResult/random/sosial-lulus/"
   
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   

    #####################  Only Demografis parameters  #####################
    print "-------------Demografis------------"
    
    
    #13. Data Mipa kelas IPK - Data Demografis
    ################################################## 
    print("Data mipa kelas ipk - Data demografis")
    
    data_train = "DataPreprocessing/data_mipa_ipk_demografis"
    data_testing = "DataPreprocessing/data_uji_mipa_ipk_dem"
    directory = "DataResult/demografis/mipa-ipk/"
   
    
    print data_train
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_, directory)
    
    
    
    #14. Data Mipa kelas Lulus - Data Demografis
    ################################################## 
    
    print("Data mipa kelas lulus - Data demografis")
    
    data_train = "DataPreprocessing/data_mipa_lulus_demografis"
    data_testing = "DataPreprocessing/data_uji_mipa_lulus_dem"
    directory = "DataResult/demografis/mipa-lulus/"
    
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   

    #15. Data Sosial kelas Ipk - Data Demografis
    ##################################################
    
    print("Data sosial kelas ipk - Data demografis")
    
    data_train = "DataPreprocessing/data_sosial_ipk_demografis"
    data_testing = "DataPreprocessing/data_uji_sosial_ipk_dem"
    directory = "DataResult/demografis/sosial-ipk/"
   
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
    

    #16. Data Sosial kelas Ipk - Data Demografis
    ################################################## 
    
    print("Data sosial kelas ipk - Data Demografis")
    
    data_train = "DataPreprocessing/data_sosial_lulus_demografis"
    data_testing = "DataPreprocessing/data_uji_sosial_lulus_dem"
    directory = "DataResult/demografis/sosial-lulus/"
   
    print data_train
    
    train,testing = load_data(data_train, data_testing)
    cls = build_tree(train,testing,directory)
    result, evaluation, temp = testing_evaluation(cls,train,testing,directory)
    summary_ = pruning_tree(directory,train, testing,temp, summary)
    re_summary(summary_,directory)
   



    
    
if __name__ == "__main__":
  
    try:
        jvm.start()
        main(sys.argv)
    except Exception, e:
        print(traceback.format_exc())
    finally:
        jvm.stop()


DEBUG:weka.core.jvm:Adding bundled jars
DEBUG:weka.core.jvm:Classpath=['c:\\python27\\lib\\site-packages\\javabridge\\jars\\rhino-1.7R4.jar', 'c:\\python27\\lib\\site-packages\\javabridge\\jars\\runnablequeue.jar', 'c:\\python27\\lib\\site-packages\\javabridge\\jars\\cpython.jar', 'c:\\python27\\lib\\site-packages\\weka\\lib\\python-weka-wrapper.jar', 'c:\\python27\\lib\\site-packages\\weka\\lib\\weka.jar']
DEBUG:weka.core.jvm:MaxHeapSize=default
DEBUG:weka.core.jvm:Package support disabled
DEBUG:javabridge.jutil:Creating JVM object
DEBUG:javabridge.jutil:Signalling caller


Data mipa kelas ipk


c:\python27\lib\site-packages\pygraphviz\agraph.py:1338: RuntimeWarning: neato: graph is too large for cairo-renderer bitmaps. Scaling by 0.663528 to fit

  warnings.warn(b"".join(errors), RuntimeWarning)
DEBUG:PIL.PngImagePlugin:STREAM 'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM 'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM 'IDAT' 59 8192


=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,592    0,055    0,617      0,592    0,604      0,546    0,852     0,536     IPK_CUM
                 0,304    0,062    0,241      0,304    0,269      0,217    0,685     0,178     IPK_KM
                 0,458    0,118    0,478      0,458    0,468      0,346    0,691     0,368     IPK_M
                 0,780    0,347    0,784      0,780    0,782      0,432    0,726     0,765     IPK_SM
Weighted Avg.    0,665    0,248    0,670      0,665    0,667      0,417    0,733     0,623     

Summary
Correctly Classified Instances         250               66.4894 %
Incorrectly Classified Instances       126               33.5106 %
Kappa statistic                          0.4067
Mean absolute error                      0.184 
Root mean squared error                  0.372 
Relative absolute error                 65.1847 %
Root relative squared 

c:\python27\lib\site-packages\pygraphviz\agraph.py:1338: RuntimeWarning: neato: graph is too large for cairo-renderer bitmaps. Scaling by 0.920163 to fit

  warnings.warn(b"".join(errors), RuntimeWarning)
DEBUG:PIL.PngImagePlugin:STREAM 'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM 'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM 'IDAT' 59 8192


...

=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,479    0,042    0,622      0,479    0,541      0,489    0,848     0,540     IPK_CUM
                 0,161    0,078    0,156      0,161    0,159      0,082    0,592     0,112     IPK_KM
                 0,359    0,156    0,319      0,359    0,338      0,194    0,683     0,277     IPK_M
                 0,753    0,420    0,747      0,753    0,750      0,335    0,703     0,748     IPK_SM
Weighted Avg.    0,603    0,299    0,610      0,603    0,605      0,310    0,709     0,590     

Summary
Correctly Classified Instances         228               60.3175 %
Incorrectly Classified Instances       150               39.6825 %
Kappa statistic                          0.2896
Mean absolute error                      0.2003
Root mean squared error                  0.3903
Relative absolute error                 70.9047 %
Root relative squared 

DEBUG:PIL.PngImagePlugin:STREAM 'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM 'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM 'IDAT' 59 8192


=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,250    0,024    0,100      0,250    0,143      0,144    0,708     0,149     LC
                 0,863    0,574    0,790      0,863    0,825      0,315    0,687     0,805     LLW
                 0,375    0,124    0,534      0,375    0,441      0,284    0,673     0,428     LTW
Weighted Avg.    0,722    0,444    0,712      0,722    0,712      0,305    0,683     0,694     

Summary
Correctly Classified Instances         273               72.2222 %
Incorrectly Classified Instances       105               27.7778 %
Kappa statistic                          0.2862
Mean absolute error                      0.2109
Root mean squared error                  0.3869
Relative absolute error                 73.9347 %
Root relative squared error            104.0434 %
Total Number of Instances              378     

__________Pruning tree with Confide

c:\python27\lib\site-packages\pygraphviz\agraph.py:1338: RuntimeWarning: neato: graph is too large for cairo-renderer bitmaps. Scaling by 0.536337 to fit

  warnings.warn(b"".join(errors), RuntimeWarning)
DEBUG:PIL.PngImagePlugin:STREAM 'IHDR' 16 13
DEBUG:PIL.PngImagePlugin:STREAM 'bKGD' 41 6
DEBUG:PIL.PngImagePlugin:'bKGD' 41 6 (unknown)
DEBUG:PIL.PngImagePlugin:STREAM 'IDAT' 59 8192


=== Detailed Accuracy By Class ===

                 TP Rate  FP Rate  Precision  Recall   F-Measure  MCC      ROC Area  PRC Area  Class
                 0,703    0,142    0,622      0,703    0,660      0,540    0,811     0,590     IPK_CUM
                 0,217    0,069    0,222      0,217    0,220      0,150    0,615     0,129     IPK_KM
                 0,371    0,077    0,245      0,371    0,295      0,243    0,751     0,190     IPK_M
                 0,639    0,393    0,713      0,639    0,674      0,241    0,646     0,686     IPK_SM
Weighted Avg.    0,603    0,283    0,620      0,603    0,609      0,308    0,691     0,584     

Summary
Correctly Classified Instances         334               60.2888 %
Incorrectly Classified Instances       220               39.7112 %
Kappa statistic                          0.3265
Mean absolute error                      0.2076
Root mean squared error                  0.389 
Relative absolute error                 72.672  %
Root relative squared 

# Summaries

## Summary data Original IPA IPK

In [ ]:
# df = pd.read_csv('WekaResultClassifier/final/kelulusan/oversampling mipa/Unpruned_TestingResult.csv')
# display(df)

df1= pd.read_csv('WekaResultClassifier/final/original/mipa-ipk/summary.csv',index_col=0)
display(df1)



## Summary data Original IPA Lulus

In [ ]:
df1= pd.read_csv('WekaResultClassifier/final/original/mipa-lulus/summary.csv',index_col=0)
display(df1)


## Summary data Original Sosial IPK

In [ ]:
df1= pd.read_csv('WekaResultClassifier/final/original/sosial-ipk/summary.csv',index_col=0)
display(df1)


## Summary data Original Sosial Lulus

In [ ]:
df1= pd.read_csv('WekaResultClassifier/final/original/sosial-lulus/summary.csv',index_col=0)
display(df1)


## Summary data Oversampling IPA IPK

In [ ]:
df1= pd.read_csv('WekaResultClassifier/final/oversampling/mipa-ipk/summary.csv',index_col=0)
display(df1)


## Summary data Oversampling IPA Lulus

In [ ]:
df1= pd.read_csv('WekaResultClassifier/final/oversampling/mipa-lulus/summary.csv',index_col=0)
display(df1)


## Summary data Oversampling Sosial IPK

In [ ]:
df1= pd.read_csv('WekaResultClassifier/final/oversampling/sosial-ipk/summary.csv',index_col=0)
display(df1)


## Summary data Sosial Sosial Lulus

In [ ]:
df1= pd.read_csv('WekaResultClassifier/final/oversampling/sosial-lulus/summary.csv',index_col=0)
display(df1)
